In [ ]:
import pandas as pd
import os
import json
from tqdm import tqdm
from pathlib import Path

# --- 설정 부분 ---
# Path 객체를 사용해서 경로를 더 안전하게 관리
BASE_DIR = Path("../")  # 상대경로
RAW_DATA_DIR = BASE_DIR / "data/raw/ai03-level1-project"
PROCESSED_DATA_DIR = BASE_DIR / "data/processed"

TRAIN_ANNO_DIR = RAW_DATA_DIR / "train_annotations"
TEST_IMAGE_DIR = RAW_DATA_DIR / "test_images"
SAVE_PATH = PROCESSED_DATA_DIR / "train_df.csv"


# --- 핵심 로직 함수 ---
def parse_raw_annotations(ann_dir: Path) -> pd.DataFrame:
    """
    복잡한 3중 폴더 구조의 원본 어노테이션을 파싱하여
    하나의 Pandas DataFrame으로 반환하는 함수.
    """
    all_annotations = []

    # Level 1: 이미지별 폴더 순회
    image_level_dirs = os.listdir(ann_dir)
    for image_dir_name in tqdm(image_level_dirs, desc="[L1] Images"):
        image_dir_path = ann_dir / image_dir_name
        if not image_dir_path.is_dir():
            continue

        # Level 2: 알약 종류 폴더 순회
        pill_level_dirs = os.listdir(image_dir_path)
        for pill_dir_name in pill_level_dirs:
            pill_dir_path = image_dir_path / pill_dir_name
            if not pill_dir_path.is_dir():
                continue

            # Level 3: 실제 .json 파일 파싱
            json_files = [f for f in os.listdir(pill_dir_path) if f.endswith(".json")]
            if not json_files:
                continue

            # 첫 번째 json 파일만 사용
            json_file_path = pill_dir_path / json_files[0]

            try:
                with open(json_file_path, "r", encoding="utf-8") as f:
                    ann_data = json.load(f)

                    image_info = ann_data.get("images", [{}])[0]
                    annotation_info = ann_data.get("annotations", [{}])[0]
                    category_info = ann_data.get("categories", [{}])[0]

                    all_annotations.append(
                        {
                            "image_id": image_info.get("id"),
                            "file_name": image_info.get("file_name"),
                            "category_id": category_info.get("id"),
                            "class_name": category_info.get("name"),
                            "bbox": annotation_info.get("bbox"),
                        }
                    )
            except Exception as e:
                print(f"\n파일 처리 에러: {json_file_path}, 에러: {e}")

    return pd.DataFrame(all_annotations)


# --- 메인 실행 부분 ---
if __name__ == "__main__":
    print("데이터 전처리를 시작합니다...")
    print(f"어노테이션 디렉토리 경로: {TRAIN_ANNO_DIR}")

# 1. 핵심 함수를 호출해서 DataFrame 생성
train_df = parse_raw_annotations(TRAIN_ANNO_DIR)

# --- (1). bbox 컬럼을 4개로 분리 (이전 단계에서 추가한 코드) ---
bbox_df = pd.DataFrame(
    train_df["bbox"].tolist(), columns=["bbox_x", "bbox_y", "bbox_w", "bbox_h"]
)
train_df = pd.concat([train_df.drop("bbox", axis=1), bbox_df], axis=1)

# --- (2). [이번 단계 추가!] category_id를 새로운 label_idx로 매핑 ---
# 고유한 category_id 목록을 뽑아 정렬
unique_category_ids = sorted(train_df["category_id"].unique())

# category_id를 0, 1, 2... 인덱스로 변환하는 딕셔너리 생성
id_to_idx = {
    int(original_id): idx for idx, original_id in enumerate(unique_category_ids)
}

# 이 매핑 정보를 사용해서 'label_idx'라는 새 컬럼을 추가
train_df["label_idx"] = train_df["category_id"].map(id_to_idx)

# 나중에 추론 결과에서 원래 클래스 이름을 찾을 수 있도록 매핑 정보도 저장
label_map = {
    "id_to_idx": id_to_idx,
    "idx_to_id": {idx: int(original_id) for original_id, idx in id_to_idx.items()},
    "id_to_name": dict(zip(train_df["category_id"], train_df["class_name"])),
}
with open(PROCESSED_DATA_DIR / "label_map.json", "w", encoding="utf-8") as f:
    json.dump(label_map, f, ensure_ascii=False, indent=4)

print(f"\n총 {len(unique_category_ids)}개의 고유 클래스를 발견했습니다.")
print("라벨 매핑 정보를 'data/processed/label_map.json'에 저장했습니다.")


# 3. 최종 DataFrame을 CSV 파일로 저장
train_df.to_csv(SAVE_PATH, index=False)

print(f"\n--- 데이터 전처리 및 저장 완료! ---")
print(train_df.head())

데이터 전처리를 시작합니다...
어노테이션 디렉토리 경로: ../data/raw/ai03-level1-project/train_annotations


[L1] Images: 100%|██████████| 498/498 [00:00<00:00, 3647.14it/s]


총 73개의 고유 클래스를 발견했습니다.
라벨 매핑 정보를 'data/processed/label_map.json'에 저장했습니다.

--- 데이터 전처리 및 저장 완료! ---
   image_id                                          file_name  category_id  \
0       477  K-002483-013395-019552-025438_0_2_0_2_90_000_2...         2482   
1       477  K-002483-013395-019552-025438_0_2_0_2_90_000_2...        25437   
2       476  K-002483-013395-019552-025438_0_2_0_2_70_000_2...        19551   
3       476  K-002483-013395-019552-025438_0_2_0_2_70_000_2...        13394   
4      1406      K-003351-033880-038162_0_2_0_2_70_000_200.png        33879   

          class_name  bbox_x  bbox_y  bbox_w  bbox_h  label_idx  
0        뮤테란캡슐 100mg     603      43     214     491          1  
1    큐시드정 31.5mg/PTP     123     810     255     256         46  
2       트루비타정 60mg/병     538     606     309     493         29  
3  써스펜8시간이알서방정 650mg     111      47     244     487         18  
4     글리틴정(콜린알포세레이트)     558     195     309     420         66  
